In [3]:
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd
import numpy as np
import os
from PIL import Image

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# import zipfile
# os.chdir(r'/content/drive/MyDrive/svarka')
# with zipfile.ZipFile('dataset.zip', 'r') as zip_obj:
#    # Extract all the contents of zip file in current directory
#    os.chdir(r'/content/drive/MyDrive/svarka')
#    zip_obj.extractall()

print('After zip extraction:')
print(os.listdir())

In [55]:
photos_names = pd.read_csv('/content/defects_table.csv')
photos_names = photos_names.drop(np.where(photos_names['target'] == 0)[0])
#photos_names['new'] = photos_names['Unnamed: 0'].astype(str) + photos_names['index']
photos_names = photos_names.drop('Unnamed: 0', axis=1)
photos_names = photos_names.drop(np.where(photos_names['index'] == "classes")[0])
photos_names.target.value_counts()

target
1    1021
Name: count, dtype: int64

In [58]:
photos_names = photos_names.drop(index= 700)

In [60]:
photos_names[(photos_names['index'] == 'classes')]

,index,target


In [61]:
photos_names.target.value_counts()

target
1    1020
Name: count, dtype: int64

In [62]:
def show_image(image):
    pil_img = Image.fromarray(image)
    pil_img= pil_img.resize((400, 400))
    return pil_img

def image_preparation(picture):
  grayscale_image = cv2.cvtColor(picture, cv2.COLOR_BGR2GRAY)
  # Удаляем шумы
  gaussian = cv2.GaussianBlur(grayscale_image, (3, 3), 121)
  #Выравниваем яркость
  image_equalize = cv2.equalizeHist(gaussian)
  return image_equalize

In [67]:
def get_model(): # prepare the model
    model = YOLO('/content/yolov8m_150epochs(best).pt') #Здесь должен быть путь к модели
    model.fuse()
    return model

In [ ]:
# путь к папке с фотками
os.chdir(r'/content/gdrive/MyDrive/svarka')

images = []
df = pd.DataFrame(columns=['filename', 'class_id', 'rel_x', 'rel_y', 'width', 'height'])
for image_path in photos_names['index']:
    image = cv2.imread(image_path + ".jpg")
    print(image_path + ".jpg")
    image = image_preparation(image)
    results = get_model()(Image.fromarray(image)) # путь к фотографии
    classes = results[0].boxes.cls.cpu().numpy()
    boxes = results[0].boxes.xywhn.cpu().numpy()
    for cls, bbox in zip(classes, boxes):
      df.loc[len(df.index)]= [image_path + ".jpg", int(cls),bbox[0],bbox[1],bbox[2],bbox[3]]

4 (79).jpg
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

0: 384x640 7 adjs, 1 geo, 1039.9ms
Speed: 5.0ms preprocess, 1039.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
4 (80).jpg
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

0: 384x640 10 adjs, 1 geo, 889.8ms
Speed: 3.0ms preprocess, 889.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
4 (81).jpg
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

0: 384x640 11 adjs, 1 geo, 1487.6ms
Speed: 4.1ms preprocess, 1487.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
4 (82).jpg
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

0: 384x640 10 adjs, 1 geo, 915.2ms
Speed: 3.2ms preprocess, 915.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
4 (83).jpg
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.

In [65]:
df

,filename,class_id,rel_x,rel_y,width,height
0,4 (79).jpg,0.0,0.674247,0.198305,0.078513,0.177306
1,4 (79).jpg,2.0,0.461395,0.810432,0.152611,0.100538
2,4 (79).jpg,0.0,0.583447,0.678115,0.029900,0.101740
3,4 (79).jpg,0.0,0.240719,0.482048,0.032094,0.166190
4,4 (79).jpg,0.0,0.553922,0.824577,0.033453,0.083692
...,...,...,...,...,...,...
3559,8 (28).jpg,2.0,0.540604,0.084077,0.201213,0.074956
3560,8 (29).jpg,2.0,0.548157,0.094165,0.189611,0.081195
3561,8 (3).jpg,2.0,0.504906,0.606567,0.285367,0.078235
3562,8 (3).jpg,3.0,0.730215,0.581596,0.111104,0.068233


In [66]:
df.to_csv('/content/submission.csv', index=False, sep=';')